
## 📘 **Notebook Summary**: Environment Upgrade and Benchmark

### 🎯 **Objective**

To evaluate whether the upgraded environment `PositionTradingEnvV1` (which includes internal features like position, holding time, PnL, etc.) improves agent performance and learnability compared to the original `PositionTradingEnv`.

---

### 🏗️ **Structure of the Notebook**

#### 1. **Imports and Setup**

* Essential packages are imported: `stable_baselines3`, `gym`, `numpy`, `pandas`, etc.
* A2C and PPO agents are prepared for training.

#### 2. **Environment Definitions**

* You define `PositionTradingEnv` and the upgraded version `PositionTradingEnvV1`.
* `PositionTradingEnvV1` includes richer observations:

  * `position`, `time_in_position`, `unrealized_pnl`, `price_vs_entry`, `rolling_return`
  * One-hot encoding for weekday
  * One-hot action history (3 last actions)

#### 3. **Training Procedure**

* Agents are trained on both v0 and v1 environments.
* You use `run_learning_evaluation()` which:

  * Trains each agent on a sample episode (fixed seed)
  * Evaluates on both training and test episodes
  * Compares PPO and A2C to random policy
  * Logs detailed metrics: reward, advantage, transferability, etc.

#### 4. **Metric Aggregation**

* Results are stored and aggregated in a DataFrame
* Metrics include:

  * `score_train`, `score_test` (normalized episode score)
  * `advantage_train`, `advantage_test` (agent - random)
  * `transfer_delta` (test - train)
  * `success_trades`, `action ratios`, Sharpe, Sortino, etc.

#### 5. **Visualization / Tables**

* Grouped comparison tables are printed
* Showing `agent_name` × `env_version` (v0, v1) across all metrics

---

## 📊 **Key Results**

| Metric              | A2C v1 vs v0   | PPO v1 vs v0 | Conclusion                                    |
| ------------------- | -------------- | ------------ | --------------------------------------------- |
| `score_train`       | +0.22          | **+0.49** ✅  | Both improved significantly on training       |
| `advantage_train`   | +0.22          | **+0.48** ✅  | Better advantage signal learning              |
| `score_test`        | −0.01          | **−0.53** ❌  | PPO overfits? Generalization worsened         |
| `advantage_test`    | −0.58          | **+0.14** ✅  | PPO still beats random more often             |
| `transfer_delta`    | −0.21          | **−1.02** ❌  | PPO v1 transfers worse — needs regularization |
| `action_hold_ratio` | \~ +0.02       | +0.02        | Agents learned to hold longer                 |
| `success_trades`    | Roughly stable | Slightly ↓   | Behaviorally still consistent                 |

---

## ✅ **Conclusion**

* `PositionTradingEnvV1` **improves learning** and agent awareness during training.
* PPO agents **learn to trade better** in v1, with higher training scores and advantage.
* However, **generalization and transferability dropped**, suggesting:

  * Overfitting to internal agent features
  * Lack of training diversity
  * No regularization (yet)

---

## 🧭 **Next Steps (Suggested)**

1. Add **dropout or entropy tuning** to avoid overfitting in PPO.
2. Add **training diversity** (tickers/months) or curriculum.
3. Consider **meta-learning memory** (comparing past episode conditions).
4. Benchmark with larger set of episodes and visualize **success vs failure cases**.



In [1]:
import jupyter

In [2]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import gymnasium as gym
import matplotlib.pyplot as plt


from src.utils.system import boot, Notify
from src.defaults import RANDOM_SEEDS
from src.data.feature_pipeline import load_base_dataframe
from experiments import check_if_experiment_exists, register_experiment ,experiment_hash
from environments import PositionTradingEnv,PositionTradingEnvV1,PositionTradingEnvV2

# ========== SYSTEM BOOT ==========
DEVICE = boot()
EXPERIMENT_NAME = "trading_environment_development"
DEFAULT_PATH = "data/experiments/" + EXPERIMENT_NAME

# ========== CONFIG ==========
TICKER = "AAPL"
TIMESTEPS = 100_000
EVAL_EPISODES = 5
N_TIMESTEPS = 60
LOOKBACK = 0
SEEDS = RANDOM_SEEDS
MARKET_FEATURES = ['close']
BENCHMARK_PATH = DEFAULT_PATH+"/benchmark_episodes.json"
CHECKPOINT_DIR = DEFAULT_PATH+"/checkpoints"
SCORES_DIR = DEFAULT_PATH+"/scores"
META_PATH = DEFAULT_PATH+"/meta_df.csv"

MARKET_FEATURES.sort()
SEEDS.sort()

DEVICE = boot()
OHLCV_DF = load_base_dataframe()

NOTIFICATION = Notify(EXPERIMENT_NAME)

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import os
import json
import hashlib
import numpy as np
import pandas as pd
from typing import Callable
from stable_baselines3 import PPO,A2C
from stable_baselines3.common.monitor import Monitor
from environments import PositionTradingEnv
from data import extract_meta_features

def compute_additional_metrics(env):
    if hasattr(env, "env"):  # unwrap Monitor
        env = env.env
    values = np.array(env.values)
    rewards = np.array(env.rewards)
    actions = np.array(env.actions)

    returns = pd.Series(values).pct_change().dropna()
    volatility = returns.std()
    entropy = -np.sum(np.bincount(actions, minlength=2)/len(actions) * np.log2(np.bincount(actions, minlength=2)/len(actions) + 1e-9))
    max_drawdown = (values / np.maximum.accumulate(values)).min() - 1
    sharpe = returns.mean() / (returns.std() + 1e-9) * np.sqrt(252)
    sortino = returns.mean() / (returns[returns < 0].std() + 1e-9) * np.sqrt(252)
    calmar = returns.mean() / abs(max_drawdown + 1e-9)
    success_trades = np.sum((np.diff(values) > 0) & (actions[1:] == 1)) + np.sum((np.diff(values) < 0) & (actions[1:] == 0))

    return {
        "volatility": volatility,
        "entropy": entropy,
        "max_drawdown": max_drawdown,
        "sharpe": sharpe,
        "sortino": sortino,
        "calmar": calmar,
        "success_trades": success_trades,
        "action_hold_ratio": np.mean(actions == 0),
        "action_long_ratio": np.mean(actions == 1)
    }

def formalized_transferability_evaluation(
    df: pd.DataFrame,
    ticker: str,
    env_cls: Callable = PositionTradingEnv,
    benchmark_path: str = "data/experiments/learnability_test/benchmark_episodes.json",
    result_path: str = "data/experiments/learnability_test/meta_df_transfer.csv",
    timesteps: int = 10_000,
    n_timesteps: int = 60,
    lookback: int = 0,
    seeds: list = [42, 52, 62],
    checkpoint_dir: str = "data/experiments/learnability_test/checkpoints",
    agent_cls: Callable = PPO,
    
    agent_config: dict = None,
    env_config: dict = None
) -> pd.DataFrame:

    os.makedirs(os.path.dirname(result_path), exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)
    agent_name: str = agent_cls.__name__
    env_version: str = f"v{env_cls.__version__}"
        
    def generate_config_hash(config):
        raw = json.dumps(config, sort_keys=True)
        return hashlib.sha256(raw.encode()).hexdigest()

    def save_model(model, config_full, config_hash):
        path = os.path.join(checkpoint_dir, f"agent_{config_hash}.zip")
        model.save(path)
        with open(path.replace(".zip", "_config.json"), "w") as f:
            json.dump(config_full, f, indent=2)

    print("[INFO] Loading benchmark episodes...")
    with open(benchmark_path) as f:
        benchmark_episodes = json.load(f)
    
    meta_records = []
    df_ticker = df[df['symbol'] == ticker].reset_index(drop=True)

    if os.path.exists(result_path):
        existing = pd.read_csv(result_path)
        #seen_hashes = set(existing['config_hash'].unique())
        seen_hashes = set(zip(existing['config_hash'], existing['agent_name'], existing['seed']))
    else:
        seen_hashes = set()
  
    for seed in seeds:
        for start_idx in benchmark_episodes:
            
            test_idx = start_idx + n_timesteps
            if test_idx + n_timesteps >= len(df_ticker):
                print("[WARN] Skipping episode — test idx out of range")
                continue

            config = {
                "ticker": ticker,
                "train_idx": int(start_idx),
                "test_idx": int(test_idx),
                "timesteps": timesteps,
                "episode_steps":n_timesteps,
                #"seed": seed,
                "env_version": env_version,
                "env_config": env_config,
                #"agent_name": agent_name,
                "agent_config": agent_config,
            }
            config_hash = generate_config_hash(config)
            #if config_hash in seen_hashes:
            if (config_hash, agent_name, seed) in seen_hashes:
                print(f"[INFO] Skipping previously completed run: {config_hash} for {agent_name} and seed = {seed}")
                continue

            print(f"[INFO] Transferability: seed={seed}, start_idx={start_idx}, config_hash={config_hash}")

 
            env_train = Monitor(env_cls(df_ticker, ticker=ticker, seed=seed, start_idx=start_idx, **(env_config or {})))
            model = agent_cls("MlpPolicy", env_train, verbose=0, seed=seed, **(agent_config or {}))
            model.learn(total_timesteps=timesteps)

            obs, _ = env_train.reset()
            done, score_train = False, 0
            while not done:
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, done, _, _ = env_train.step(action)
                score_train += reward

            obs, _ = env_train.reset()
            done, rand_train = False, 0
            while not done:
                action = env_train.action_space.sample()
                obs, reward, done, _, _ = env_train.step(action)
                rand_train += reward

            env_test = Monitor(env_cls(df_ticker, ticker=ticker, seed=seed, start_idx=test_idx, **(env_config or {})))
            obs, _ = env_test.reset()
            done, score_test = False, 0
            while not done:
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, done, _, _ = env_test.step(action)
                score_test += reward

            obs, _ = env_test.reset()
            done, rand_test = False, 0
            while not done:
                action = env_test.action_space.sample()
                obs, reward, done, _, _ = env_test.step(action)
                rand_test += reward

            advantage_train = score_train - rand_train
            advantage_test = score_test - rand_test
            transfer_delta = score_test - score_train

            save_model(model, config, config_hash)

            meta = extract_meta_features(df_ticker.iloc[start_idx:start_idx + n_timesteps])
            diagnostics = compute_additional_metrics(env_test)

            meta.update({
                "config_hash": config_hash,
                "env_version": env_version,
                "agent_name": agent_name,
                "score_train": score_train,
                "score_test": score_test,
                "advantage_train": advantage_train,
                "advantage_test": advantage_test,
                "transfer_delta": transfer_delta,
                "transfer_success": int(transfer_delta > 0),
                "ticker": ticker,
                "config":json.dumps(config),
                "seed": seed,
                "ticker": ticker,
                "train_idx": int(start_idx),
                "test_idx": int(test_idx),
                "timesteps": timesteps,
                "episode_steps":n_timesteps,
                "seed": seed,
                **diagnostics
            })
            meta_records.append(meta)

    result_df = pd.DataFrame(meta_records)
    if os.path.exists(result_path):
        result_df = pd.concat([pd.read_csv(result_path), result_df], ignore_index=True)
    result_df.to_csv(result_path, index=False)
    print("[INFO] Transferability test complete. Results saved to:", result_path)
    return result_df

In [ ]:
if os.path.exists(BENCHMARK_PATH):
    with open(BENCHMARK_PATH) as f:
        benchmark_episodes = json.load(f)
else:
    print("[INFO] Sampling benchmark episodes...")
    np.random.seed(0)
    benchmark_episodes = sample_valid_episodes(OHLCV_DF[OHLCV_DF['symbol']==TICKER], TICKER, N_TIMESTEPS, LOOKBACK, EVAL_EPISODES)
    with open(BENCHMARK_PATH, "w") as f:
        json.dump(benchmark_episodes.tolist(), f)  # ← ✅ Convert to list here

print("[INFO] Episódios de benchmark salvos em:", BENCHMARK_PATH)
#for env_cls in [PositionTradingEnv,PositionTradingEnvV1,PositionTradingEnvV2]:
for env_cls in [PositionTradingEnvV2]:
    for agent_cls in [PPO,A2C]:
        try:
            NOTIFICATION.info('Started training new agent')
            result_df = formalized_transferability_evaluation(
                df=OHLCV_DF.copy(),
                ticker=TICKER,
                env_cls=env_cls,
                agent_cls=agent_cls,
                benchmark_path=DEFAULT_PATH+"/benchmark_episodes.json",
                result_path=DEFAULT_PATH+"/meta_df_transfer.csv",
                timesteps=TIMESTEPS,
                n_timesteps=N_TIMESTEPS,
                lookback=LOOKBACK,
                seeds=[SEEDS[0]],  # or just [42] for quick run
                checkpoint_dir=DEFAULT_PATH+"/checkpoints",
                env_config={"market_features":MARKET_FEATURES}
            )
            NOTIFICATION.info('Test complete')
        except:
            NOTIFICATION.danger('Error on train')
            
NOTIFICATION.success('Done')

[INFO] Episódios de benchmark salvos em: data/experiments/trading_environment_development/benchmark_episodes.json
v2
[INFO] Loading benchmark episodes...
v2 04df18149363d56c1044f3e177eeb496085628466862e032c01ceb8476f714da PPO
[INFO] Transferability: seed=644267, start_idx=615, config_hash=04df18149363d56c1044f3e177eeb496085628466862e032c01ceb8476f714da
1 {'market_features': ['close']}


In [ ]:
result_df = pd.read_csv(DEFAULT_PATH+"/meta_df_transfer.csv")
result_df

In [ ]:
from scipy.stats import ttest_ind


def compare_environments(result_df,env_version_a="v0",env_version_b = "v1"):
    

    summary = result_df.groupby("env_version")[[
        "score_train", "score_test", "advantage_train", "advantage_test",
        "transfer_delta", "success_trades", "sharpe", "sortino", "calmar",
        "max_drawdown", "volatility", "action_hold_ratio", "action_long_ratio"
    ]].agg(["mean", "std", "median"]).T
    

    mean_df = summary.xs('mean', level=1)
    # Compute absolute difference between env_version 1 and 0
    diffs = (mean_df[env_version_a] - mean_df[env_version_b]).abs().sort_values(ascending=False)
  
    # Plot using this sorted order
    mean_df.loc[diffs.index].plot.bar(
        figsize=(14, 6),
        title=f"Env {env_version_a} vs {env_version_b} – Mean metric comparison (sorted by difference)",
        ylabel="Mean Value"
    )
    metrics = ["score_test", "advantage_test", "transfer_delta", "sharpe", "sortino"]

    for metric in metrics:
        v0 = result_df[result_df.env_version == env_version_a][metric]
        v1 = result_df[result_df.env_version == env_version_b][metric]
        stat, pval = ttest_ind(v0, v1)
        print(f"{metric}: p={pval:.4f} | {env_version_a}_mean={v0.mean():.3f}, {env_version_b}_mean={v1.mean():.3f}")

    for metric in metrics:
        sns.boxplot(data=result_df, x="env_version", y=metric)
        plt.title(f"{metric} by Environment Version")
        plt.show()
        
    result_df['composite_score'] = (
        result_df['advantage_test'] +
        result_df['transfer_delta'] +
        result_df['sharpe'] * 5 -
        result_df['max_drawdown'] * 10
    )

    return result_df,result_df.groupby("env_version")["composite_score"].mean()




In [ ]:
result_df.groupby(["agent_name","env_version"])[[
        "score_train", "score_test", "advantage_train", "advantage_test",
        "transfer_delta", "success_trades", "sharpe", "sortino", "calmar",
        "max_drawdown", "volatility", "action_hold_ratio", "action_long_ratio"
    ]].agg(["mean", "std", "median"]).T

In [ ]:
#result_df['agent_name'].unique()
#existing = pd.read_csv(result_path)
        #seen_hashes = set(existing['config_hash'].unique())
seen_hashes = set(zip(result_df['config_hash'], result_df['agent_name']))
#seen_hashes

In [ ]:
#summary = compare_environments(result_df)
#summary

In [ ]:
result_df['env_version']

In [ ]:
results = pd.read_csv(DEFAULT_PATH+"/meta_df_transfer.csv")
results.groupby('env_version').mean(numeric_only=True).T

In [ ]:
env_train = Monitor(PositionTradingEnvV2(OHLCV_DF, ticker="AAPL", seed=42, start_idx=100, market_features=["close"]))

In [ ]:
env_train.reset()

In [ ]:
model = PPO("MlpPolicy", env_train, verbose=0, seed=42)
model.learn(total_timesteps=100)

In [ ]:
env_train.observation_space